# Sequence Classification using Recurrent Neural Networks(RNN)


In [1]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt
import torch.optim as optim

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: True


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).




In [2]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size,
       sampler=DD.sampler.RandomSampler(trSet),
       num_workers=2, pin_memory=False)
valLD = DD.DataLoader(valSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(valSet),
       num_workers=1, pin_memory=False)
tstLD = DD.DataLoader(tstSet, batch_size=batch_size,
       sampler=DD.sampler.SequentialSampler(tstSet),
       num_workers=1, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

In [3]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
    
        self.project_layer = nn.Linear(75, 100)
        self.recurrent_layer = nn.LSTM(100, 100, num_layers=1)
        self.classify_layer = nn.Linear(100, 10)
      
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        logits = self.classify_layer(rnn_outputs[:,-1])
        return logits

model = SequenceClassify()

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [4]:

model.cuda() # For GPU use
optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().cuda()

In [5]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 50
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3062,  Accuracy: 10.00


/home/h4k3r/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


val Epoch: 0  , Loss: 2.3045,  Accuracy: 9.75
train Epoch: 1  , Loss: 2.3059,  Accuracy: 10.15
train Epoch: 2  , Loss: 2.3049,  Accuracy: 9.90
train Epoch: 3  , Loss: 2.3046,  Accuracy: 9.65
train Epoch: 4  , Loss: 2.3042,  Accuracy: 10.05
train Epoch: 5  , Loss: 2.3035,  Accuracy: 10.30
val Epoch: 5  , Loss: 2.3025,  Accuracy: 9.75
train Epoch: 6  , Loss: 2.3038,  Accuracy: 10.50
train Epoch: 7  , Loss: 2.3032,  Accuracy: 10.00
train Epoch: 8  , Loss: 2.3031,  Accuracy: 10.05
train Epoch: 9  , Loss: 2.3024,  Accuracy: 10.35
train Epoch: 10  , Loss: 2.3030,  Accuracy: 9.95
val Epoch: 10  , Loss: 2.3016,  Accuracy: 10.00
train Epoch: 11  , Loss: 2.3025,  Accuracy: 10.15
train Epoch: 12  , Loss: 2.3022,  Accuracy: 10.10
train Epoch: 13  , Loss: 2.3023,  Accuracy: 10.15
train Epoch: 14  , Loss: 2.3022,  Accuracy: 10.25
train Epoch: 15  , Loss: 2.3017,  Accuracy: 10.65
val Epoch: 15  , Loss: 2.3011,  Accuracy: 11.00
train Epoch: 16  , Loss: 2.3019,  Accuracy: 10.80
train Epoch: 17  , Loss:

In [10]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
    
        
        self.project_layer = nn.Linear(75,100)
        self.recurrent_layer = nn.LSTM(100, 200, num_layers=2, dropout=0.5, batch_first = True)
        self.recurrent_layer2 = nn.LSTM(200, 400, num_layers=4, dropout=0.5, batch_first = True)
        self.classify_layer1 = nn.Linear(400, 20)
        self.classify_layer2 = nn.Linear(20, 10)
       
        

    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        rnn_outputs, (hn, cn) = self.recurrent_layer2(rnn_outputs)
        logits = self.classify_layer1(rnn_outputs[:,-1])
        logits = self.classify_layer2(logits)
        return logits


In [11]:

model = SequenceClassify()
model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().cuda()
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.data[0])
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 1000
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


/home/h4k3r/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train Epoch: 0  , Loss: 2.3112,  Accuracy: 9.85
val Epoch: 0  , Loss: 2.2919,  Accuracy: 12.00
train Epoch: 1  , Loss: 2.3059,  Accuracy: 10.10
train Epoch: 2  , Loss: 2.3042,  Accuracy: 9.25
train Epoch: 3  , Loss: 2.2992,  Accuracy: 10.15
train Epoch: 4  , Loss: 2.1516,  Accuracy: 18.05
train Epoch: 5  , Loss: 1.8200,  Accuracy: 29.75
val Epoch: 5  , Loss: 1.7288,  Accuracy: 30.25
train Epoch: 6  , Loss: 1.6123,  Accuracy: 39.80
train Epoch: 7  , Loss: 1.5043,  Accuracy: 43.95
train Epoch: 8  , Loss: 1.4577,  Accuracy: 46.55
train Epoch: 9  , Loss: 1.3615,  Accuracy: 49.90
train Epoch: 10  , Loss: 1.2096,  Accuracy: 56.25
val Epoch: 10  , Loss: 1.1239,  Accuracy: 60.50
train Epoch: 11  , Loss: 1.1571,  Accuracy: 57.60
train Epoch: 12  , Loss: 1.1577,  Accuracy: 57.70
train Epoch: 13  , Loss: 1.0083,  Accuracy: 63.25
train Epoch: 14  , Loss: 1.0049,  Accuracy: 63.85
train Epoch: 15  , Loss: 1.0283,  Accuracy: 62.20
val Epoch: 15  , Loss: 1.1155,  Accuracy: 61.75
train Epoch: 16  , Los

Process Process-257:
Process Process-256:
Traceback (most recent call last):
  File "/home/h4k3r/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/h4k3r/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/h4k3r/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/h4k3r/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/h4k3r/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/h4k3r/anaconda3/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/h4k3r/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", lin

KeyboardInterrupt: 

In [ ]:
# Use your best model to generate results on test set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('resultsss.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)